# Inferenza

### In Locale

In [ ]:
from keras.models import load_model
import numpy as np
import json

In [ ]:
model = load_model("abalone_model.keras")

In [ ]:
predict_input = np.array([[0.475,0.37,0.125,0.5095,0.2165,0.1125,0.165,9]])
predict_result = model.predict(predict_input)

json.dumps({"predict_result": predict_result.tolist()})

In [ ]:
import scipy.sparse
from sagemaker import image_uris
image_uris.retrieve(framework='tensorflow',region='us-east-1',version='2.12.1',image_scope='inference',instance_type='ml.c5.4xlarge')

Quella prodotta dalla cella sopra sarà la URL dell'immagine da usare all'intero del Dockerfile.train

In [ ]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-1.amazonaws.com

In [ ]:
!docker build -t inference-housing-container -f Dockerfile.inference .

In [ ]:
!docker images

In [ ]:
%%sh
# Specify an image name
image_name=tensorflow-inference
echo "image_name: ${image_name} ######################"

account=$(aws sts get-caller-identity --query Account --output text)
echo "account: ${account} ######################"

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
echo "region: ${region} ######################"

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image_name}:2.12.1-cpu"
echo "fullname: ${fullname} ######################"

aws ecr describe-repositories --repository-names "${image_name}"
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name "${image_name}"
fi

In [ ]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 891377019371.dkr.ecr.us-east-1.amazonaws.com

In [ ]:
!docker tag inference-housing-container 891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.12.1-cpu

In [ ]:
!docker push 891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.12.1-cpu

In [ ]:
from sagemaker import get_execution_role
import boto3
import datetime
from time import gmtime, strftime

# https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-deployment.html

my_session = boto3.session.Session()
aws_region = my_session.region_name

sagemaker_client = boto3.client('sagemaker', region_name=aws_region)

sagemaker_role = get_execution_role()

model_name = 'housing-price-predictor'

# Create model
create_model_response = sagemaker_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = sagemaker_role,
    PrimaryContainer = {
        'Image': '891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.12.1-cpu',
        'ModelDataUrl': 's3://itsarghisdata/data/output/tensorflow-training-2024-11-21-22-26-58-248/output/model.tar.gz',
    })

In [ ]:
# Create an endpoint config name. Here we create one based on the date  
# so it we can search endpoints based on creation time.
endpoint_config_name = 'housing-inference'

instance_type = 'ml.p2.xlarge'

endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    # You will specify this name in a CreateEndpoint request.
    # List of ProductionVariant objects, one for each model that you want to host at this endpoint.
    ProductionVariants=[
        {
            "VariantName": "variant1", # The name of the production variant.
            "ModelName": model_name,
            "InstanceType": instance_type, # Specify the compute instance type.
            "InitialInstanceCount": 1 # Number of instances to launch initially.
        }
    ]
)

print(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")

# The name of the endpoint. The name must be unique within an AWS Region in your AWS account.
endpoint_name = 'inference-endpoint'

create_endpoint_response = sagemaker_client.create_endpoint(
                                            EndpointName=endpoint_name, 
                                            EndpointConfigName=endpoint_config_name)